### open csv files from directory and create markdown tables

- get csv files from the local build input/image/source folder
- print out markdown files
- save files as input/includes files
- !check out how I made excel files...

In [1]:
from pathlib import Path
import pandas as pd
import csv
from json import loads

In [2]:
# get path from IG output dir
in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/images/data-element-mapping.csv')
# in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/includes/ballot-intro.csv')
out_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/test_resources/')

in_path

PosixPath('/Users/ehaas/Documents/FHIR/davinci-ecdx/input/images/data-element-mapping.csv')

In [3]:
                                  
# load CSV using pandas

df = pd.read_csv(in_path)
df.fillna('-', inplace=True)
df

,No,Data Element,X12n 277,X12n 278 Response,X12n 275,CDex #submit-attachment Parameter,CDex Request Attachment Task Profile Element,Comments
0,1,Tracking ID (Provider or Payer Assigned),REF02,-,-,TrackingId,Task.identifier,Provider or Payer assigned tracking control nu...
1,2,Use,-,-,-,AttachTo,Task.reasonCode,"Choice of ""claim"" or ""preauthorization"""
2,3,Payer ID,NM108,-,-,PayerId,Task.reasonReference.identifier,Payer ID
3,4,Payer URL,-,-,-,(operation endpoint),"""payer-url"" Task.input",Payer endpoint where the attachments are submi...
4,5,Organization ID,-,-,-,OrganizationId,-,Organization of Provider who submitted claim/p...
5,6,Provider ID,-,-,-,ProviderId,Task.owner.identifier,Provider who submitted claim/prior authorization
6,7,Claim/PreAuth ID,-,-,-,-,Task.reasonReference.identifier,claim/prior authorization ID (Provider or Paye...
7,8,Line Item(s),-,-,-,Attachment.LineItem,“code” Task.input.extension,claim/prior authorization line item numbers
8,9,LOINC Attachment Code,STC01-02,-,-,Attachment.Code,“code” Task.input,LOINC attachment codes
9,10,Due Date,DPT02,-,-,-,Task.restriction.period,Deadline form submitting attachments to Payer


In [8]:

# convert to markdown table 
# select columns:

maps = {'data_elements_to_submit' : ([0,1,3,4,5,7,8,10,11],[1,5,7]),
        'attachments_to_275' : ([0,1,3,4,5,7,8,10,11],[1,5,4,7]),
        'attachments_to_requests_277_278' : ([0,1,3,4,5,7,8,10,11],[5,6,2,3]),
        'attachments_to_requests' : ([0,1,3,4,5,7,8,10,11],[1,5,6]),
        'data_elements_to_requests': (None,[1,6,7]),
        'requests-277_278': (None,[1,6,2,3,7])
        }
# maps = {'text' : ([0,1,3,4,5,7,8,10,11],[1,6,5]) }
# maps = {'text' : (None,[0,1,2]) }

df1 = df[['No','Data Element','CDex #submit-attachment Parameter', 'X12n 275']]
df1 = df.iloc[:, [0,1,5,4,7]]
for k,v in maps.items():
  if k != 'text':

    my_file =f'{k}.md'
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    with open(out_path / my_file, 'w') as md:
      my_table = df1.to_markdown(index=False, tablefmt="github")
      for i in range(1,10):
        my_table = my_table.replace('----','---')
        my_table = my_table.replace('  ',' ')
      md.write(my_table + '\n{:.grid}')
  else:
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    my_table = df1.to_markdown(index=False, tablefmt="github")
    for i in range(1,10):
      my_table = my_table.replace('----','---')
      my_table = my_table.replace('  ',' ')
    print(my_table)
